In [11]:
import nbimporter
from prepare.preprocessing import process_all_scans
from prepare.patch_extraction import extract_patches_from_annotations, extract_negative_patches_from_candidates
from prepare.build_training_index import build_training_index, create_balanced_training_csv
from prepare.classes import LunaPatchDataset, Advanced3DAugment
from torch.utils.data import random_split, DataLoader

In [3]:
LUNA_PATH = r"C:\Users\azizd\python\pytorch\data\bitirme_projesi\archive"
PREPROCESSED_OUTPUT = "./output/preprocessed_luna16"
ANNOTATION_FILE = LUNA_PATH+r"\annotations.csv"
CANDIDATES_FILE = LUNA_PATH+r"\candidates_V2\candidates_V2.csv"
PATCH_OUTPUT = "./output/patches"
TRAINING_FILE = "./output/training_balanced.csv"

In [ ]:
# --- Stage 1: Preprocess All Scans ---
process_all_scans(LUNA_PATH, PREPROCESSED_OUTPUT)

In [ ]:
# --- Stage 2: Extract 3D Patches for Training ---
extract_patches_from_annotations(
    annotation_csv=ANNOTATION_FILE,
    metadata_csv=f"{PREPROCESSED_OUTPUT}/preprocessed_metadata.csv",
    output_folder=PATCH_OUTPUT,
    patch_size=32
)

In [5]:
extract_negative_patches_from_candidates(
    candidates_csv=CANDIDATES_FILE,
    annotations_csv=ANNOTATION_FILE,
    metadata_csv=f"{PREPROCESSED_OUTPUT}/preprocessed_metadata.csv",
    output_folder=PATCH_OUTPUT,
    patch_size=32
)

Saved negative patch: ./output/patches\1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_274_252_132_neg.npy
Saved negative patch: ./output/patches\1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_61_219_65_neg.npy
Saved negative patch: ./output/patches\1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_85_261_283_neg.npy
Saved negative patch: ./output/patches\1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_67_225_333_neg.npy
Saved negative patch: ./output/patches\1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_126_237_111_neg.npy
Saved negative patch: ./output/patches\1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_64_202_346_neg.npy
Saved negative patch: ./output/patches\1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_106_222_242_neg.npy
Saved negative patch: ./output/patches\1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260_136_289_292_neg.npy
Saved negativ

KeyboardInterrupt: 

In [4]:
# --- Stage 3: Build the Balanced Training Index CSV ---
create_balanced_training_csv(
    patch_folder=PATCH_OUTPUT,
    output_csv=TRAINING_FILE,
    oversample_pos=True,
    downsample_neg=True
)

Original: 112 positive, 2143 negative
Balanced dataset saved: ./output/training_balanced.csv
Final counts → Positive: 112, Negative: 112


In [12]:
# --- Stage 5: Augmentation & DataLoader ---
augment_adv = Advanced3DAugment()

train_dataset = LunaPatchDataset(
    TRAINING_FILE,
    transform=augment_adv
)

In [13]:
# --- Stage 6: Split Into Train/Validation ---
val_percent = 0.2
val_size = int(len(train_dataset) * val_percent)
train_size = len(train_dataset) - val_size

train_ds, val_ds = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)